# 实验 3.2：PGD 迭代对抗攻击

## 🎯 学习目标

完成本实验后，你将能够：
- ✅ 解释 PGD（投影梯度下降）的迭代攻击原理
- ✅ 对比 FGSM 单步攻击与 PGD 多步攻击的效果差异
- ✅ 实现带投影约束的迭代优化算法
- ✅ 分析迭代次数对攻击成功率和收敛速度的影响

## 📚 前置知识

- 完成实验 3.1：FGSM 白盒攻击
- 理解梯度下降优化算法
- 了解投影操作和约束优化

## 🖥️ 实验环境

| 项目 | 说明 |
|------|------|
| 平台 | 腾讯 Cloud Studio |
| GPU | NVIDIA Tesla T4（16GB）|
| 模型 | ResNet-18（ImageNet 预训练）|
| 框架 | PyTorch 2.0+ |

## 📝 填空说明

本实验共 **4 个填空练习**，难度：⭐⭐⭐☆☆

## 核心概念回顾

**PGD（Projected Gradient Descent）** 是 FGSM 的迭代增强版本，由 Madry 等人提出，被认为是最强的一阶对抗攻击方法。

**FGSM vs PGD 对比**：

| 特性 | FGSM | PGD |
|------|------|-----|
| 步数 | 单步 | 多步迭代 |
| 强度 | 较弱 | 更强 |
| 速度 | 快 | 慢 |
| 精确度 | 粗糙 | 精细 |

**PGD 算法流程**：

```
初始化：x_0 = x（或 x + 随机噪声）
对于 t = 1 到 T：
    1. 计算梯度：g_t = ∇_x Loss(θ, x_t, y)
    2. 梯度步进：x_t+1 = x_t + α × sign(g_t)
    3. 投影到约束域：x_t+1 = Proj_{x+S}(x_t+1)
    4. 裁剪像素值：x_t+1 = clip(x_t+1, 0, 1)
返回：x_T
```

**关键参数**：
- `ε`：最大扰动范围（L∞ 范数约束）
- `α`：每步步长（通常 α < ε）
- `T`：迭代次数

**投影操作**：确保扰动不超过 ε 限制
```python
Proj_{x+S}(x') = clip(x' - x, -ε, ε) + x
```

## 第一部分：环境准备与依赖安装

In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

# 加载预训练模型
print("=" * 50)
print("🔧 环境准备")
print("=" * 50)
print("正在加载 ResNet-18 模型...")
model = models.resnet18(pretrained=True)
model.eval()
print("✓ 模型加载成功！")

# ImageNet 标准化参数
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

print(f"  PyTorch 版本: {torch.__version__}")
print("=" * 50)

In [ ]:
# 创建测试图像（与实验 3.1 相同，保持一致性）
def create_test_image():
    """创建用于测试的合成图像"""
    np.random.seed(42)
    img = np.random.rand(224, 224, 3) * 0.3 + 0.35
    center_x, center_y = 112, 112
    for i in range(224):
        for j in range(224):
            dist = np.sqrt((i - center_x)**2 + (j - center_y)**2)
            if dist < 60:
                img[i, j] = [0.1, 0.1, 0.1]  # 黑色中心
            elif dist < 80:
                img[i, j] = [0.9, 0.9, 0.9]  # 白色边缘
    return torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)

def predict(model, img_tensor):
    """预测图像类别和置信度"""
    input_tensor = normalize(img_tensor).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)
    probs = torch.softmax(output, dim=1)
    pred_class = output.argmax(dim=1).item()
    confidence = probs[0, pred_class].item()
    return pred_class, confidence

# 创建原始图像
original_image = create_test_image()
original_class, original_confidence = predict(model, original_image)

# 显示原始图像和预测
plt.figure(figsize=(5, 5))
plt.imshow(original_image.permute(1, 2, 0).numpy())
plt.title(f"原始测试图像\n预测类别: {original_class}, 置信度: {original_confidence:.2%}", fontsize=12)
plt.axis('off')
plt.tight_layout()
plt.show()

print("=" * 50)
print("📊 原始预测结果")
print("=" * 50)
print(f"  预测类别: {original_class}")
print(f"  预测置信度: {original_confidence:.2%}")
print("=" * 50)

## 第二部分：实现 PGD 迭代攻击算法

PGD 的核心是"多次小步前进 + 投影约束"。我们将逐步实现这个算法。

In [ ]:
def pgd_attack(model, image, label, epsilon, alpha, num_steps):
    """
    实现 PGD (Projected Gradient Descent) 迭代对抗攻击
    
    参数:
        model: 目标神经网络模型
        image: 原始图像张量 [C, H, W]，范围 [0, 1]
        label: 原始标签（整数）
        epsilon: 最大扰动范围 ε（L∞ 约束）
        alpha: 每步的步长 α
        num_steps: 迭代次数 T
    
    返回:
        adversarial_image: 对抗样本张量 [C, H, W]
    """
    # 步骤1：初始化对抗样本
    # 策略A：从原图开始（常用）
    adv_image = image.clone().detach()
    # 策略B：从原图+随机噪声开始（更强）
    # adv_image = image + torch.empty_like(image).uniform_(-epsilon, epsilon)
    # adv_image = torch.clamp(adv_image, 0, 1)
    
    # 步骤2：开始迭代优化
    for step in range(num_steps):
        # 2.1 启用梯度计算
        adv_image.requires_grad = True
        
        # 2.2 前向传播
        normalized = normalize(adv_image).unsqueeze(0)
        output = model(normalized)
        
        # 2.3 计算损失
        loss = nn.CrossEntropyLoss()(output, torch.tensor([label]))
        
        # 2.4 反向传播计算梯度
        model.zero_grad()
        loss.backward()
        
        # 2.5 获取梯度
        gradient = adv_image.grad.data
        
        # ========== 填空 1：沿梯度方向前进一小步 ==========
        # 
        # 🎯 任务：实现 PGD 的单步更新（类似 FGSM）
        # 
        # 💡 提示：
        #   - PGD 每步的更新公式：x_t+1 = x_t + α × sign(梯度)
        #   - 使用 alpha 作为步长（不是 epsilon）
        #   - 记得先 detach() 断开计算图
        # 
        # 📖 参考：
        #   FGSM 是一步到位：x_adv = x + ε × sign(梯度)
        #   PGD 是多步小跳：x_t+1 = x_t + α × sign(梯度)
        # 
        # 难度：⭐⭐☆☆☆
        # 
        # 请将 ___________ 替换为正确的代码
        
        adv_image = ___________  # 期望：adv_image.detach() + alpha * torch.sign(gradient)
        
        # ========== 填空 2：投影到 ε-ball 约束域 ==========
        # 
        # 🎯 任务：确保扰动不超过 epsilon 限制
        # 
        # 💡 提示：
        #   - 计算扰动：perturbation = adv_image - image
        #   - 使用 torch.clamp() 裁剪扰动到 [-epsilon, epsilon]
        #   - 投影公式：perturbation = clamp(perturbation, -ε, ε)
        # 
        # 难度：⭐⭐⭐☆☆
        # 
        # 请将 ___________ 替换为正确的代码
        
        perturbation = adv_image - image
        perturbation = ___________  # 期望：torch.clamp(perturbation, -epsilon, epsilon)
        
        # 2.7 应用投影后的扰动
        adv_image = image + perturbation
        
        # 2.8 裁剪到有效像素范围 [0, 1]
        adv_image = torch.clamp(adv_image, 0, 1)
    
    return adv_image.detach()

print("✓ PGD 攻击函数定义完成！")

In [ ]:
# 同时实现 FGSM 用于对比
def fgsm_attack(model, image, label, epsilon):
    """FGSM 单步攻击（用于对比）"""
    image_copy = image.clone().detach()
    image_copy.requires_grad = True
    
    normalized = normalize(image_copy).unsqueeze(0)
    output = model(normalized)
    loss = nn.CrossEntropyLoss()(output, torch.tensor([label]))
    
    model.zero_grad()
    loss.backward()
    
    gradient = image_copy.grad.data
    perturbation = epsilon * torch.sign(gradient)
    adversarial_image = torch.clamp(image_copy + perturbation, 0, 1)
    
    return adversarial_image.detach()

print("✓ FGSM 对比函数定义完成！")

## 第三部分：FGSM vs PGD 效果对比

使用相同的 ε 值，对比单步 FGSM 和多步 PGD 的攻击效果。

In [ ]:
# 设置攻击参数
epsilon = 0.03      # 最大扰动范围
alpha = 0.01        # PGD 每步步长（通常设置为 ε/步数 或更小）
num_steps = 10      # PGD 迭代次数

print("=" * 50)
print("⚔️  FGSM vs PGD 攻击对比")
print("=" * 50)
print(f"  攻击参数: ε={epsilon}, α={alpha}, 步数={num_steps}")
print("=" * 50)

# 执行 FGSM 攻击（单步）
fgsm_adv = fgsm_attack(model, original_image, original_class, epsilon)
fgsm_pred, fgsm_conf = predict(model, fgsm_adv)

# ========== 填空 3：执行 PGD 攻击 ==========
# 
# 🎯 任务：调用 pgd_attack 函数执行多步迭代攻击
# 
# 💡 提示：
#   - 参数：model, original_image, original_class, epsilon, alpha, num_steps
#   - 所有参数已在上面定义好
# 
# 难度：⭐☆☆☆☆
# 
# 请将 ___________ 替换为正确的代码

pgd_adv = ___________  # 期望：pgd_attack(model, original_image, original_class, epsilon, alpha, num_steps)

pgd_pred, pgd_conf = predict(model, pgd_adv)

# 显示对比结果
print(f"\n原始图像:")
print(f"  预测: 类别 {original_class}, 置信度: {original_confidence:.2%}")
print(f"\nFGSM 攻击 (1步):")
print(f"  预测: 类别 {fgsm_pred}, 置信度: {fgsm_conf:.2%}")
print(f"  攻击结果: {'✓ 成功' if fgsm_pred != original_class else '✗ 失败'}")
print(f"\nPGD 攻击 ({num_steps}步):")
print(f"  预测: 类别 {pgd_pred}, 置信度: {pgd_conf:.2%}")
print(f"  攻击结果: {'✓ 成功' if pgd_pred != original_class else '✗ 失败'}")
print("=" * 50)

In [ ]:
# 可视化对比
fig, axes = plt.subplots(1, 4, figsize=(18, 5))

# 子图1：原始图像
axes[0].imshow(original_image.permute(1, 2, 0).numpy())
axes[0].set_title(f"原始图像\n类别 {original_class}\n置信度: {original_confidence:.1%}", 
                  fontsize=11, color='green')
axes[0].axis('off')

# 子图2：FGSM 对抗样本
axes[1].imshow(fgsm_adv.permute(1, 2, 0).numpy())
fgsm_color = 'red' if fgsm_pred != original_class else 'blue'
axes[1].set_title(f"FGSM (1步)\n类别 {fgsm_pred}\n置信度: {fgsm_conf:.1%}", 
                  fontsize=11, color=fgsm_color)
axes[1].axis('off')

# 子图3：PGD 对抗样本
axes[2].imshow(pgd_adv.permute(1, 2, 0).numpy())
pgd_color = 'red' if pgd_pred != original_class else 'blue'
axes[2].set_title(f"PGD ({num_steps}步)\n类别 {pgd_pred}\n置信度: {pgd_conf:.1%}", 
                  fontsize=11, color=pgd_color)
axes[2].axis('off')

# 子图4：扰动大小对比
fgsm_perturbation = (fgsm_adv - original_image).abs().mean().item()
pgd_perturbation = (pgd_adv - original_image).abs().mean().item()

bars = axes[3].bar(['FGSM', 'PGD'], [fgsm_perturbation, pgd_perturbation], 
                   color=['orange', 'purple'])
axes[3].set_title('平均扰动大小对比', fontsize=11)
axes[3].set_ylabel('扰动值')
axes[3].set_ylim(0, max(fgsm_perturbation, pgd_perturbation) * 1.2)

# 添加数值标签
for bar in bars:
    height = bar.get_height()
    axes[3].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontsize=10)

plt.suptitle("FGSM vs PGD 攻击效果对比", fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

print(f"\n扰动分析:")
print(f"  FGSM 平均扰动: {fgsm_perturbation:.6f}")
print(f"  PGD 平均扰动:  {pgd_perturbation:.6f}")

### 🤔 思考一下

1. **攻击强度**：PGD 和 FGSM 哪个攻击成功率更高？为什么？
2. **扰动大小**：两种方法的扰动大小有什么差异？
3. **实际应用**：在什么场景下会选择 FGSM？什么场景下选择 PGD？

## 第四部分：探索迭代次数的影响

迭代次数是 PGD 的重要参数。我们将测试不同迭代次数对攻击效果的影响。

In [ ]:
# 测试不同迭代次数的效果
step_values = [1, 3, 5, 10, 20, 40]
epsilon = 0.03
alpha = 0.01

print("=" * 75)
print("🔬 不同迭代次数的攻击效果")
print("=" * 75)
print(f"{'迭代次数':<10} {'预测类别':<12} {'预测置信度':<15} {'原类别置信度':<15} {'攻击结果'}")
print("-" * 75)

results = []
for steps in step_values:
    # ========== 填空 4：对不同迭代次数执行 PGD 攻击 ==========
    # 
    # 🎯 任务：对每个迭代次数执行 PGD 攻击
    # 
    # 💡 提示：
    #   - 参数：model, original_image, original_class, epsilon, alpha, steps
    #   - steps 是当前循环变量
    # 
    # 难度：⭐⭐☆☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    adv = ___________  # 期望：pgd_attack(model, original_image, original_class, epsilon, alpha, steps)
    
    pred, conf = predict(model, adv)
    
    # 计算原类别的置信度（用于观察攻击效果）
    with torch.no_grad():
        output = model(normalize(adv).unsqueeze(0))
        probs = torch.softmax(output, dim=1)
        orig_class_conf = probs[0, original_class].item()
    
    success = "✓ 成功" if pred != original_class else "✗ 失败"
    results.append((steps, adv, pred, conf, orig_class_conf))
    print(f"{steps:<10} {pred:<12} {conf:<15.2%} {orig_class_conf:<15.2%} {success}")

print("=" * 75)

In [ ]:
# 可视化迭代次数的影响
fig = plt.figure(figsize=(16, 10))

# 子图1：原类别置信度随迭代次数的变化
ax1 = plt.subplot(2, 2, 1)
steps_list = [r[0] for r in results]
orig_conf_list = [r[4] for r in results]

ax1.plot(steps_list, orig_conf_list, 'bo-', linewidth=2, markersize=8, label='原类别置信度')
ax1.axhline(y=original_confidence, color='g', linestyle='--', linewidth=2, 
            label=f'初始置信度 ({original_confidence:.1%})')
ax1.axhline(y=0.5, color='r', linestyle='--', linewidth=1, alpha=0.5, label='50% 阈值')
ax1.set_xlabel('迭代次数', fontsize=11)
ax1.set_ylabel('原类别置信度', fontsize=11)
ax1.set_title('迭代次数 vs 原类别置信度', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 子图2：新预测类别的置信度
ax2 = plt.subplot(2, 2, 2)
new_conf_list = [r[3] for r in results]
ax2.plot(steps_list, new_conf_list, 'ro-', linewidth=2, markersize=8, label='新预测类别置信度')
ax2.set_xlabel('迭代次数', fontsize=11)
ax2.set_ylabel('新类别置信度', fontsize=11)
ax2.set_title('迭代次数 vs 新预测置信度', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 子图3-6：展示不同迭代次数的对抗样本
selected_indices = [0, 2, 4, 5]  # 选择 1, 5, 20, 40 步
for i, idx in enumerate(selected_indices):
    ax = plt.subplot(2, 4, 5 + i)
    steps, img, pred, conf, _ = results[idx]
    ax.imshow(img.permute(1, 2, 0).numpy())
    color = 'red' if pred != original_class else 'green'
    ax.set_title(f"{steps} 步\n类别: {pred}\n置信度: {conf:.1%}", 
                 fontsize=10, color=color)
    ax.axis('off')

plt.suptitle(f"PGD 迭代次数影响分析 (ε={epsilon}, α={alpha})", 
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

## 第五部分：PGD 攻击过程可视化

深入观察 PGD 在每一步迭代中如何逐步降低原类别的置信度。

In [ ]:
# 实现带历史记录的 PGD 攻击
def pgd_attack_with_history(model, image, label, epsilon, alpha, num_steps):
    """
    PGD 攻击并记录每一步的中间结果
    
    返回:
        adversarial_image: 最终对抗样本
        history: 每步的历史记录 [(step, pred_class, pred_conf, orig_class_conf), ...]
    """
    history = []
    adv_image = image.clone().detach()
    
    for step in range(num_steps):
        adv_image.requires_grad = True
        normalized = normalize(adv_image).unsqueeze(0)
        output = model(normalized)
        loss = nn.CrossEntropyLoss()(output, torch.tensor([label]))
        
        model.zero_grad()
        loss.backward()
        
        gradient = adv_image.grad.data
        adv_image = adv_image.detach() + alpha * torch.sign(gradient)
        
        # 投影操作
        perturbation = adv_image - image
        perturbation = torch.clamp(perturbation, -epsilon, epsilon)
        adv_image = torch.clamp(image + perturbation, 0, 1)
        
        # 记录当前状态
        pred, conf = predict(model, adv_image)
        with torch.no_grad():
            out = model(normalize(adv_image).unsqueeze(0))
            probs = torch.softmax(out, dim=1)
            orig_conf = probs[0, label].item()
        history.append((step + 1, pred, conf, orig_conf))
    
    return adv_image.detach(), history

# 执行 PGD 并记录过程
print("执行 PGD 攻击并记录每步变化...")
_, history = pgd_attack_with_history(model, original_image, original_class, 
                                      epsilon=0.03, alpha=0.01, num_steps=30)

# 提取数据
steps = [h[0] for h in history]
orig_confs = [h[3] for h in history]
new_preds = [h[1] for h in history]
new_confs = [h[2] for h in history]

# 可视化攻击过程
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 左图：原类别置信度下降曲线
ax1.plot(steps, orig_confs, 'b-o', linewidth=2, markersize=4, label='原类别置信度')
ax1.axhline(y=original_confidence, color='g', linestyle='--', linewidth=2, 
            label=f'初始置信度 ({original_confidence:.1%})')
ax1.axhline(y=0.5, color='r', linestyle='--', linewidth=1, alpha=0.5)
ax1.fill_between(steps, orig_confs, alpha=0.3)
ax1.set_xlabel('迭代步数', fontsize=12)
ax1.set_ylabel('原类别置信度', fontsize=12)
ax1.set_title('PGD 攻击过程：原类别置信度变化', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 右图：预测类别变化
pred_changes = []
current_pred = original_class
for step, pred in enumerate(new_preds):
    if pred != current_pred:
        pred_changes.append((step + 1, current_pred, pred))
        current_pred = pred

ax2.plot(steps, new_confs, 'r-o', linewidth=2, markersize=4, label='新预测类别置信度')
ax2.set_xlabel('迭代步数', fontsize=12)
ax2.set_ylabel('新类别置信度', fontsize=12)
ax2.set_title('PGD 攻击过程：新预测类别置信度变化', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 输出统计信息
print("=" * 50)
print("📊 PGD 攻击过程分析")
print("=" * 50)
print(f"  初始置信度: {original_confidence:.2%}")
print(f"  最终置信度: {orig_confs[-1]:.2%}")
print(f"  置信度下降: {(original_confidence - orig_confs[-1]):.2%}")
print(f"  预测类别变化次数: {len(pred_changes)}")
if pred_changes:
    print(f"  首次改变预测: 第 {pred_changes[0][0]} 步 (类别 {pred_changes[0][1]} → {pred_changes[0][2]})")
print("=" * 50)

## 📋 实验小结

### 核心收获

通过本实验，你已经：

1. **理解原理**：掌握了 PGD 迭代攻击的算法原理和实现方法
2. **对比分析**：深入理解了 PGD 相比 FGSM 的优势和代价
3. **参数调优**：分析了迭代次数对攻击效果的影响
4. **过程可视化**：观察了 PGD 如何逐步降低模型对原类别的置信度

### 关键概念回顾

**PGD vs FGSM 核心区别**：

| 特性 | FGSM | PGD |
|------|------|-----|
| 更新公式 | `x_adv = x + ε·sign(∇L)` | `x_t+1 = Proj(x_t + α·sign(∇L))` |
| 步数 | 1 | T (多步) |
| 投影操作 | 无 | 有（确保 ‖x_adv - x‖∞ ≤ ε）|
| 攻击强度 | 较弱 | 更强 |
| 计算成本 | 低 | 高（T倍于FGSM）|

**PGD 三个关键步骤**：
1. **梯度步进**：x_t+1 = x_t + α × sign(∇L)
2. **投影约束**：Proj(x_t+1) 到 ε-ball
3. **像素裁剪**：clip(x_t+1, 0, 1)

**参数选择建议**：
- **ε**：总扰动预算（如 0.03, 0.1）
- **α**：步长，通常设为 ε/10 或 2.5×ε/T
- **T**：迭代次数，通常 10-40 步

### 实验观察

1. **攻击强度**：PGD 通常比 FGSM 有更高的攻击成功率
2. **收敛性**：PGD 在前几步快速降低原类别置信度，后续收益递减
3. **计算成本**：PGD 需要 T 次前向传播和反向传播，是 FGSM 的 T 倍
4. **应用场景**：
   - **FGSM**：快速攻击、对抗训练（降低计算成本）
   - **PGD**：强力攻击、模型鲁棒性评估、生成高质量对抗样本

### 填空参考答案

<details>
<summary>点击查看所有填空参考答案</summary>

**填空 1：沿梯度方向前进一小步**
```python
adv_image = adv_image.detach() + alpha * torch.sign(gradient)
```

**填空 2：投影到 ε-ball 约束域**
```python
perturbation = torch.clamp(perturbation, -epsilon, epsilon)
```

**填空 3：执行 PGD 攻击**
```python
pgd_adv = pgd_attack(model, original_image, original_class, epsilon, alpha, num_steps)
```

**填空 4：对不同迭代次数执行 PGD 攻击**
```python
adv = pgd_attack(model, original_image, original_class, epsilon, alpha, steps)
```

</details>

### 延伸学习

- **下一个实验**：实验 3.3 - 迁移攻击（黑盒攻击场景）
- **论文阅读**：Madry et al., "Towards Deep Learning Models Resistant to Adversarial Attacks" (ICLR 2018)
- **进阶挑战**：
  - 实现 Targeted PGD（攻击到指定目标类别）
  - 尝试随机初始化版本的 PGD
  - 对比 L2 范数约束的 PGD

### 🤔 深入思考

1. **收敛分析**：从置信度变化曲线看，PGD 是否一定收敛？
2. **参数权衡**：如何在攻击强度和计算成本之间找到平衡？
3. **防御思路**：基于 PGD 的观察，你能想到哪些防御方法？
4. **实际应用**：在黑盒场景（攻击者不知道模型参数）下，PGD 还能使用吗？

### 安全提醒

⚠️ **本实验仅用于教育目的**

PGD 是评估模型鲁棒性的标准方法，也是对抗训练的重要工具。请将所学知识用于：
- 评估和提升 AI 系统的安全性
- 开发更鲁棒的模型
- 推进对抗机器学习研究

请勿用于攻击未经授权的系统。

⚠️ 重要提醒

实验完成后，请记得：

1. **释放 GPU 资源**：如果你使用的是 Tencent CloudStudio，请点击右上角的"停止"按钮，停止运行。
2. **保存实验结果**：如需保存，请下载 notebook 文件

这样可以避免不必要的资源占用和费用产生。**否则会一直消耗你的免费资源额度。**

In [ ]:
# 实验结束，清理显存（可选）
# 如果要继续进行下一个实验，可以运行这段代码释放显存

del model
import torch
torch.cuda.empty_cache()
print("✓ 显存已清理")